In [2]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')
# get pandas
import pandas as pd
# we'll do some visual checks, get the necessary packages
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
# done, save the model
import pickle

In [4]:
# first load the model
model = pickle.load(open('../models/elastic_net_v1.pkl', 'rb'))

In [36]:
# now load the test data
df = pd.read_csv('../data/test.csv')

In [37]:
# prep the feature columns
input_features = ['GarageArea', 'TotalBsmtSF', '1stFlrSF', 'YearBuilt']
target = ['SalePrice']
cols = ['Id'] + input_features
df = df[cols]

In [38]:
# take care of na
# use 1stFlrSF for bsmt when null
df.loc[df['TotalBsmtSF'].isnull(),'TotalBsmtSF'] = df['1stFlrSF']
# similar for garage but add factor 1/2
df.loc[df['GarageArea'].isnull(),'GarageArea'] = df['1stFlrSF']*0.5
df.describe()

,Id,GarageArea,TotalBsmtSF,1stFlrSF,YearBuilt
count,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,2190.000000,472.767649,1046.015079,1156.534613,1971.357779
std,421.321334,216.974170,442.764155,398.165820,30.390071
min,1461.000000,0.000000,0.000000,407.000000,1879.000000
25%,1825.500000,318.000000,784.000000,873.500000,1953.000000
50%,2190.000000,480.000000,988.000000,1079.000000,1973.000000
75%,2554.500000,576.000000,1304.000000,1382.500000,2001.000000
max,2919.000000,1488.000000,5095.000000,5095.000000,2010.000000


In [40]:
# prep scikit
X = df[input_features].values

In [41]:
y = model.predict(X)

In [42]:
# predict given input
df['log_SalePrice'] = y

In [43]:
# get the original variable 
df['SalePrice'] = np.exp(df['log_SalePrice']) - 1

In [45]:
# drop the log var
df = df.drop(['log_SalePrice'], axis=1)

In [46]:
df.head()

,Id,GarageArea,TotalBsmtSF,1stFlrSF,YearBuilt,SalePrice
0,1461,730.0,882.0,896,1961,169634.220691
1,1462,312.0,1329.0,1329,1958,158662.154054
2,1463,482.0,928.0,928,1997,170368.012545
3,1464,470.0,926.0,926,1998,169644.263991
4,1465,506.0,1280.0,1280,1992,197920.060147


In [51]:
# save the file for submission
df[['Id','SalePrice']].to_csv('../data/test_submission.csv',index=False)